In [12]:
import os
from google.colab import drive
# Montamos la unidad de drive
drive.mount('/content/drive')
project_path = '/content/drive/MyDrive/RNA_trabajo_2'

# Chequeamos el directorio de trabajo
if os.path.exists(project_path):
    # Cambiamos el directorio de trabajo
    os.chdir(project_path)
    print(f"Directorio de trabajo cambiado a: {os.getcwd()}")
else:
    print(f"Error: La carpeta '{project_path}' no existe en Google Drive después del montaje.")
    print("Por favor, verifica la ruta y asegúrate de que la carpeta esté en tu Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directorio de trabajo cambiado a: /content/drive/MyDrive/RNA_trabajo_2


<center>

# **Curso: Redes Neuronales Artificiales y Algoritmos Bioinspirados**

---

## ***Semestre*: 2025 - 1**

---

## ***Trabajo #2: Redes Neuronales Aplicadas a Datos Tabulares***

---

### **Equipo #2**:

Carlos José Quijano Valencia
<br>
Miller Alexis Quintero García
<br>
Kelly Yojana Ospina Correa
<br>
Mateo Sebastián Mora Montero
<br>
Stiven Julio Doval

<br>

**Profesor:** Juan David Ospina Arango  
**Monitor:** Andrés Mauricio Zapata Rincón

<br>

**Universidad Nacional de Colombia, Sede Medellín**

***Mayo 2 del 2025***

**Enlace a repositorio GitHub:** <https://github.com/Straver00/RNA_2025-1>

</center>

# Tabla de contenidos

1.  [Introducción](#introducción)
2.  [Delimitación y Metodología](#delimitación-metodología)
    -   [sub2.1](#sub2.1)
    -   [sub2.2](#sub2.2)
        - [sub2.2.1](#sub2.2.1)
        - [sub2.2.2](#sub2.2.2)
    -   [sub2.3](#sub2.3)
        - [sub2.3.1](#sub2.3.1)
3.  [Limpieza de datos](#limpieza-datos)
4.  [Análisis Descriptivo e Hipótesis](#análisis-hipótesis)
    -   [sub4.1](#sub4.1)
    -   [sun4.2](#sub4.2)
5.  [Modelos](#modelos)
    - [Definición de Métricas Claves](#métricas)
    - [Tratamiento del Desbalance](#desbalance)
    - [Función de Pérdida](#función-perdida)
    - [Estrategia de Entrenamiento](#estrategia-entrenamiento)
    - [Perceptrones Simples como Referencia](#perceptrones)
        - [Learning Rate 0.001](#learning-rate-1)
        - [Learning Rate 0.01](#learning-rate-2)
        - [Comparación Perceptrones](comparación-perceptrones)
    - [Redes Neuronales Sencillas](#redes-neuronales)
        - [Sin Dropout](#sin-dropout)
        - [Con Dropout](#con-dropout)
        - [Comparación Redes Sencillas](#comparación-redes)
    - [Red con Batch Normalization y Dropout](#red-batch-dropout)
        - [Métricas en test](#métricas-test)
    - [Selección de umbral de corte](#umbral)
        - [Comparación métricas tras el umbral](#comparación)
        - [Matrices de confusión](#matriz_confusión)
        - [Discusión Mejor Modelo](#mejor-modelo)
    - [Caso de uso del modelo](#caso-uso)
6.  [Aplicación web](#app-web)
7.  [Bibliografía](#bibliografía)
8.  [Reporte de Contribución Individual](#reporte-de-contribución-individual)

# Introducción

# Delimitación y Metodología
El objetivo de desarrollar el modelo se plantea en un contexto en cuál se debe desarrollar una app web para uso de personas. En ese sentido la delimitación inmediata que surge es que el app web debe tener un formulario, en el que la persona ingrese sus datos y así, conozca tanto su scorecard, como si puede o no acceder a crédito.
Considerando eso, lo primero será entender que el modelo debe recibir información y variables que una persona si pueda proporcionar fácilmente desde su conocimiento, y sin necesidad de pedir información que sea condifencial de entidades bancarias o burós de crédito.
Tras esa consideración, se entiende entonces que hay muchas variables que se podrán descarta.

Así pues un flujo estrategico a seguir es:

- Limpieza de datos, descartando variables con criterios de imposibilidad de preguntar o no factibilidad de imputación por enormes proporciones de datos nulos.
- Codificación de variables categóricas.
- Análisis para obtener indicadores e ideas valiosas el dataset.
- Escalamiento de variables numéricas continuas.
- División de los datos en entrenamiento y prueba.
- Entrenamiento de diversos modelos con técnicas variadas.
- Selección del mejor modelo en base a métricas relacionadas con el problema de clasificación.


# Limpieza de datos

# Análisis Descriptivo e Hipótesis

# Modelos
Para el desarrollo de modelos predictivos se empleó las interfaz de alto nivel proporcionada por la librerías Keras, y usando PyTorch como backend.

Antes de empezar a entrenar modelos es importante definir métricas acorde a la naturaleza de este problema de clasificación de personas como potenciales no pagadores de crédito. Además utilizar alguna estrategía para tratar el fuerte desbalanace de clases evidenciado en la anterior sección.

## Definición de Métricas Claves

Este problema de clasificación binaria trata sobre predecir en base a la probabilidad de impago retornada por un modelo, si una persona que va solicitar crédito califica como pagador o no pagador. La variable objetivo se codificó como **1 para un cliente que no pagó**, por lo tanto lo ideal es tener una tasa de detección de verdaderos positivos muy alta (True Positive Rate), dicha métrica también se conoce como *Recall* o *Sensitivity*. Esta será la métrica clave ya que el dinero que se pierde por no prestarle a un cliente que no pagará, es mucho mayor que el que se pierde por no prestarle a un cliente que si pagará.

\begin{equation}
\text{Recall} = \frac{TP}{TP + FN}
\end{equation}  

A parte de esto, también vamos a usar como métrica el **AUC PR** para comparar los modelos entre sí, para ver el mejor clasificador, esta es mejor que la **AUC ROC**, ya que el **AUC PR** determina el área bajo la curva entre el **Precision** y el **Recall** para diversos umbrales, dicha métrica se compara respecto a un *baseline* determinado por la proporción de muestras de la variable objetivo **Y = 1** (no pagadores), así pues es mucho mejor indicador en problemas desbalanceados.

La escogencia entonces del modelo de despliegue estará sujeta a una comparativa de:

- Recall/Sensitivity (True Positive Rate)
- AUC PR (área bajo la curva PR)
- AUC ROC (área bajo la curva ROC)
- Accuracy (exactitud general de clasificación)

## Tratamiento del Desbalance

Siendo un desbalance claro como se vió en la sección de análisis, con aproximadamente 78.09% de los datos siendo de la clase 0 (si pagó), y 21.90% para la clase 1 (no pagó); es necesario abordar técnicas para tratar este desbalance de clases.
Aquí la estrategia elegida fue usar una técnica de ponderación de la función de pérdida, para indicarle a los modelos que presten atención a las muestras de la clase subrepresentada. Esta se implementó fácilmente con la función `class_weight` de la librería Scikit-Learn, la cuál por medio de las etiquetas, estudia su proporción y calcula los factores de ponderación para la función de pérdida.

## Función de Pérdida
Elegimos como función de pérdida la entropía binaria cruzada por su idoneidad natural para problemas de clasificación binaria cuando se combina con la activación sigmoide en la capa de salida. Esta función está fundamentada en principios de máxima verosimilitud, ya que modela la salida de la red como una distribución de Bernoulli, donde cada predicción representa la probabilidad de que una muestra pertenezca a la clase positiva. La BCE penaliza de manera asimétrica las predicciones incorrectas, siendo más severa con aquellas predicciones que muestran alta confianza pero son erróneas, lo cual es deseable desde el punto de vista del aprendizaje.
Además, la BCE presenta propiedades matemáticas óptimas para el entrenamiento de redes neuronales: es convexa cuando se usa con la activación sigmoide, garantizando la existencia de un mínimo global, y sus derivadas son suaves y bien comportadas, facilitando la convergencia del algoritmo de optimización. Su implementación es computacionalmente eficiente y numéricamente estable, evitando problemas comunes como el desvanecimiento del gradiente que pueden surgir con otras funciones de pérdida en este tipo de problemas.

$$\text{BCE} = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \log(\hat{y}_i) + (1 - y_i) \log(1 - \hat{y}_i) \right]$$

Donde:

- $N$ es el número de muestras

- $y_i$ es la etiqueta verdadera (0 o 1) para la muestra

- $\hat{y}_i$ es la probabilidad predicha para la muestra

## Estrategia de Entrenamiento
Para el entrenamiento vamos a emplearon diversas estrategias variadas, unas se aplicaran a unos modelos y otras no. Entre todas estas posibles estrategias están:

- Diferentes learning rates.
- Implementar split de datos, para tener un subconjunto de validación, esto de forma aleatoria en cada época.
- Uso de Model CheckPoints para guardar mejores parámetros en una época según los resultados en el split de validación.
- Uso de Early Stoppings para detener el entrenamiento cuando según alguna métrica, el modelo no ha mejorado tras cierta cantidad de épocas.
- Capas de Dropout como estrategia de regularización y añadir ruido al entrenamiento para quizás mejorar el desempeño con datos con diferentes distribución y/o evitar el overfitting.
- Capas de Batch Normalization para estabilizar y acelerar el entrenamiento. Esto reduce el desplazamiento de covariables internas, lo que permite usar tasas de aprendizaje más altas y hace que el entrenamiento sea menos sensible a la inicialización de pesos.

## Perceptrones Simples como Referencia
Una buen práctica es siempre iniciar con modelos de baja complejidad para tener un referencia o pistas que permitan trazar un camino de hacia donde orientar el diseño de arquitectura.
Para esto se van a entrenar 2 perceptrones con diferentes learning rate (uno con 0.001 y otro con 0.01), y ambos van usar Model CheckPoint para guardar los pesos de la época con el mejor **Recall en el split de validación**. Dando entonces lugar a 4 modelos diferentes de perceptrones simples.

La arquitectura de todos es la misma, una capa de entrada que recibe las características escaladas y codificadas, con una única neurona de salida con función de activación sigmoide para obtener la probabilidad.

Veamos entonces los resultados de estos modelos para el conjunto de prueba de los datos.

![Comparativa de métricas de perceptrones](MyDrive/RNA_trabajo_2/images_report/comparativa_metricas_perceptrones.png)

.... Ahora veamos las curvas ROC de los modelos

<img src="images_report/curvas_roc_perceptrones.png" alt="Fig 21. Curvas ROC Perceptrones">

Pasamos ahora a ver las curvas PR (Precission - Recall) ya que estas son más apropidas para clasificadores con datos desbalanceados

<img src="images_report/curvas_pr_perceptrones.png" alt="Fig 22. Curvas PR Perceptrones">

Tras analizar los resultados de los 4 perceptrones, y considerar que métricas como **AUC ROC** y **AUC PR** indican en general que tan bueno es el clasificador (para todos los umbrales), la elección del perceptrón de referencia final, estará sujeta a eso, siendo entonces que los perceptrones tipo 2 (learning rate de 0.01), tienen mejor puntaje en este caso, por lo cual se seleccionará entre los de este tipo.
Ahora bien, entre estos 2, el perceptrón con parámetros guardados por checkpoint en la mejor época es el que tiene mejor *recall (true positive rate)*, la cual es la métrica clave. Por tal motivo:

**Perceptrón seleccionado como modelo de referencia**: Perceptrón 2 Checkpoint.


## Redes Neuronales Sencillas

Aquí se plantearon arquitecturas más complejas con el ánimo de captar mejor el comportamientos de los datos de entrenamiento.

La arquitectura es de 3 capas ocultas con función de activación **LeakyReLU** en cada neuronas, la razón de esta función de activación es para poder aprovechar los valores negativos en los datos de entrenamiento, cosa que nos perderiamos si se usará la conocida **ReLU**.

Por otro lado la cantidad de neuronas por capa seguirán una estrategia de arquitectura piramidal, siguiendo un flujo de número de neuronas así:

n_features $\rightarrow$ 2 $\cdot$ n_features $\rightarrow$ n_features $\rightarrow$ n_features/2 $\rightarrow$ 1

Teniendo al final una función de activación sigmoide con una única neurona para obtener las probabilidades.

Se plantearon 2 variaciones de esta arquitectura, una **sin Dropout** y la otra **con capas de Dropout de 0.2**

Además, aquí se repetirá de nuevo la estrategia de utilizar Model CheckPoints para guardar los parámetros de la red en la época